In [ ]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

cudnn.benchmark = True #-- uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms.
                       #-- If this is set to false, uses some in-built heuristics that might not always be fastest.

cudnn.fastest = True #-- this is like the :fastest() mode for the Convolution modules,
                     #-- simply picks the fastest convolution algorithm, rather than tuning for workspace size

from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.optim as optim

import random, os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from models.resnet import *
from utils import *

In [ ]:
import ConfigParser
Config = ConfigParser.ConfigParser()
Config.read("config.ini")
# path to the dataset
path_to_FOURVIERE_CLEAN2 = Config.get("Corpus", 'path_to_FOURVIERE_CLEAN2')

In [ ]:
imSize = 225
batchSize = 32
nb_epoch = 50

In [ ]:
# Load train and test data

pathToImg = path_to_FOURVIERE_CLEAN2+"IMAGE_FROM_V/"
# Dictionary with the imgName as key and corresponding class as value for the test and train part
ImgNameToClassTrain = {line.split(' ')[1]:line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_train.txt").read().splitlines()}
ImgNameToClassTest = {line.split(' ')[1]:line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_test_requestInTrain.txt").read().splitlines()}
# Convert the class to a numeric id
ClassToIdClass = {Artworks:i for i, Artworks in enumerate(sorted(set([line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_train.txt").read().splitlines()])))}
# Associate imgName to the corresponding numeric class id
ImgNameToIdClassTrain = {ImgName:ClassToIdClass[ImgNameToClassTrain[ImgName]] for ImgName in  ImgNameToClassTrain.keys()}
ImgNameToIdClassTest = {ImgName:ClassToIdClass[ImgNameToClassTest[ImgName]] for ImgName in  ImgNameToClassTest.keys()}

# open all images
dImgTrain = {imgName:Image.open(pathToImg+imgName) for imgName in ImgNameToClassTrain.keys()}
dImgTest = {imgName:Image.open(pathToImg+imgName).resize((imSize, imSize), Image.BILINEAR) for imgName in ImgNameToClassTest.keys()}

In [ ]:
# Compute mean and standard deviation for images normalisation
mean = ComputeMean(dImgTrain.values())
std = ComputeStdDev(dImgTrain.values())

imageTrainTransform = transforms.Compose([
    transforms.Scale(300), 
    transforms.RandomCrop(225), 
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean,
                         std = std),
])

imageTestTransform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean,
                         std = std),
])

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# defined a new net with the number of classes corresponding to the dataset
ResnetFineTuned = resnet50(len(ImgNameToIdClassTrain)).train()
# load the pre-trained model and copy only the Features 
copyFeaturesParametersResnet(test, resnet50(pretrained=True), 3, 4, 6, 3, "Bottleneck")
ResnetFineTuned = ResnetFineTuned.cuda()

In [ ]:
#define the optimizer to train only the fully connected layer lr = 0.01
optimizer=optim.SGD([{'params': ResnetFineTuned.fc.parameters()},
                     {'params': ResnetFineTuned.conv1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.bn1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer2.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer3.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer4.parameters(), 'lr': 0.0},
                    ], lr=0.01, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(ResnetFineTuned, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(ResnetFineTuned, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)    

In [ ]:
torch.save(ResnetFineTuned, "ckpt/ResnetFineTuned-50epoch-lr-0.01"".ckpt")

In [ ]:
#define the optimizer to train only the fully connected layer lr = 0.001
optimizer=optim.SGD([{'params': ResnetFineTuned.fc.parameters()},
                     {'params': ResnetFineTuned.conv1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.bn1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer2.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer3.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer4.parameters(), 'lr': 0.0},
                    ], lr=0.001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(ResnetFineTuned, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(ResnetFineTuned, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)    

In [ ]:
torch.save(ResnetFineTuned, "ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")

In [ ]:
ResnetFineTuned = torch.load("ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")
ResnetFineTuned = ResnetFineTuned.cuda()

In [ ]:
# load the model and define the optimizer to train net with lr of 0.001
optimizer=optim.SGD(ResnetFineTuned.parameters(), lr=0.001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(ResnetFineTuned, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(ResnetFineTuned, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)    

In [ ]:
torch.save(ResnetFineTuned, "ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epochAll-lr-0.001.ckpt")

In [ ]:
# load the model and define the optimizer to train net with lr of 0.0001
optimizer=optim.SGD(ResnetFineTuned.parameters(), lr=0.0001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(ResnetFineTuned, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(ResnetFineTuned, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)    

In [ ]:
torch.save(ResnetFineTuned, "ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epochAll-lr-0.001-50epochAll-lr-0.0001.ckpt")

In [ ]:
# load the model and define the optimizer to train net with lr of 0.0001
optimizer=optim.SGD(ResnetFineTuned.parameters(), lr=0.00001, momentum=0.9)
for epoch in range(50): # loop over the dataset multiple times
    print "epoch", epoch, 
    trainclassifier(optimizer)
    testClassifier()

In [ ]:
torch.save(ResnetFineTuned, "ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epochAll-lr-0.001-50epochAll-lr-0.0001-50epochAll-lr-0.00001.ckpt")